In [33]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import time
#from numba import jit, autojit

<h1> Loading data

In [34]:
pathhome = os.getcwd()
layer_name = 'fc6'
pathcnn = pathhome+'\\outcome\\imagenet_vgg_f\\'+layer_name

<h2> Loading csv files with photo ids/business ids and sample submussion 

In [35]:
# Business ids and photo ids train data set
df_train_photo_ids = pd.read_csv('train_photo_to_biz_ids.csv')
# Business ids and photo ids test data set
df_test_photo_ids = pd.read_csv('test_photo_to_biz.csv')

<h2> Loading CNNcodes from images

In [36]:
def getCNNcodes(matfile, path):
    data = sio.loadmat(path+'\\'+matfile)
    CNNcodes = []
    photoids = []
    for d in data['data']:
        photoid = (d[0][0][0][0][0][:-4])
        photoids.append(photoid)
        CNNcode = np.array(d[0][0][0][1].flatten())
        CNNcodes.append(CNNcode)
    return photoids, CNNcodes

In [37]:
def duplicate_images(CNNcodes):    
    seen = set()
    dubs = []
    for c in CNNcodes:
        cstr = np.array(c).mean()
        if cstr not in seen:
            seen.add(cstr)
        else:
            dubs.append(cstr)
        del cstr
    CNNcodes_nodubs = [c for c in CNNcodes if np.array(c).mean() not in dubs]
    return np.array(CNNcodes_nodubs)            

<h3> Train data

In [46]:
list_chunks = np.array_split(os.listdir(pathcnn+"\\train"),50)

In [56]:
def meanCNN_for_chunk(chunk, df_photo_ids, path):
    df_CNNcodes =pd.DataFrame({'photo_id': [],'CNNcode': []})
    for f in chunk:
        photoids, CNNcodes = getCNNcodes(f,path)
        df = pd.DataFrame({'photo_id': photoids,'CNNcode': CNNcodes})
        df_CNNcodes =pd.concat([df_CNNcodes, df])

    df_CNNcodes = df_CNNcodes.convert_objects(convert_numeric = True)
    
    df_photo_biz_codes = pd.merge(df_photo_ids, df_CNNcodes, on='photo_id')
    grouped = df_photo_biz_codes.groupby('business_id')
    df = grouped.aggregate(lambda x: list(x))
    df.drop('photo_id', axis=1, inplace=True)
    
    df["meanCNN"] = ""
    df["num_of_instances"] = ""

    for i,r in df.iterrows():
        CNNcode = np.array(r['CNNcode'])
        #CNNcode = duplicate_images(CNNcodes)
        meanCNN = CNNcode.mean(axis=0)
        df.loc[i, 'meanCNN'] = meanCNN
        df.loc[i, 'num_of_instances'] = len(np.array(r['CNNcode']))
        
    df.drop('CNNcode', axis=1, inplace = True)
    
    return df

In [186]:
def testrow(r):
    ninst = np.sum(np.array(r['num_of_instances']))
    inst = [float(x)/ninst for x in r['num_of_instances']]
    CNNcodes = r['meanCNN']
    C = map(lambda x,y: x*y, CNNcodes, inst)
    C = np.array(C).sum(axis = 0)
    r['num_of_instances'] = ninst
    r['meanCNN'] = C
    return r

def mergerunningmean(df1, df2):
    df12 = pd.concat([df1, df2], axis =0)
    grouped = df12.groupby('business_id')
    df12g = grouped.aggregate(lambda x: list(x))
    df12g.apply(testrow, axis=1)
    return df12g

In [ ]:
df1

In [194]:
i =1
flag = 0
for chunk in list_chunks:
    print(i)
    df = meanCNN_for_chunk(chunk, df_train_photo_ids, pathcnn+"\\train")
    df.reset_index(inplace=True)
    if flag == 1:
        df_train = mergerunningmean(df_train, df) 
    else:
        df_train = df 
        flag = 1
        
    i=i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [198]:
i =1
flag = 0
df_list = []
for chunk in list_chunks:
    print(i)
    df = meanCNN_for_chunk(chunk, df_train_photo_ids, pathcnn+"\\train")
    df.reset_index(inplace=True)
    df_list.append(df)
    i=i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [203]:
df_train_photo_ids.groupby('business_id').aggregate('count')

,photo_id
business_id,
3,43
4,83
5,410
6,84
7,21
8,34
9,75
12,22
13,61


In [204]:
df_all.head(5)

,business_id,meanCNN,num_of_instances
0,4,"[0.0, 5.98449, 0.0, 0.0, 17.2075, 0.0, 0.0, 0....",1
1,5,"[2.7932, 4.46373, 3.10721, 6.41132, 2.67737, 2...",6
2,6,"[2.15894, 2.07827, 8.18354, 8.61245, 8.89241, ...",3
3,7,"[0.0, 0.0, 0.0, 0.0, 8.38736, 0.0, 0.0, 0.0, 0...",1
4,8,"[26.8714, 0.0, 4.56455, 0.0, 0.0, 2.24279, 10....",1


In [205]:
df_all = pd.concat(df_list,axis=0)
grouped = df_all.groupby('business_id')
df12g = grouped.aggregate(lambda x: list(x))
df12g.apply(testrow, axis=1)

,meanCNN,num_of_instances
business_id,,
3,"[3.97935, 3.8044, 0.0490416, 7.39866, 11.3186,...",43
4,"[7.8422, 6.29956, 2.25657, 3.65727, 5.91263, 2...",84
5,"[5.13244, 6.49476, 1.83091, 6.13681, 14.4489, ...",410
6,"[8.39003, 7.00344, 4.83912, 4.78227, 9.12754, ...",84
7,"[5.11108, 5.19684, 0.0, 8.02328, 11.249, 5.484...",21
8,"[3.62037, 3.82908, 2.80725, 5.07891, 14.2267, ...",34
9,"[10.7781, 4.57718, 0.167967, 8.3709, 3.06552, ...",75
12,"[10.474, 7.42825, 3.17585, 7.63761, 14.2271, 4...",22
13,"[7.18588, 6.76315, 1.89236, 4.70331, 9.75041, ...",61


In [ ]:
df12g.head(15)

In [ ]:
name = 'df_train_'+layer_name
df12g.to_pickle(name)

In [ ]:
df_CNNcodes_train =pd.DataFrame({'photo_id': [],'CNNcode': []})

# Extracting CNNcodes and photoids for train images
for f in os.listdir(pathcnn+"\\train"):
    photoids, CNNcodes = getCNNcodes(f,pathcnn+"\\train")
    df = pd.DataFrame({'photo_id': photoids,'CNNcode': CNNcodes})
    df_CNNcodes_train =pd.concat([df_CNNcodes_train, df])
    
df_CNNcodes_train = df_CNNcodes_train.convert_objects(convert_numeric = True)

In [ ]:
df_train_photo_biz_codes = pd.merge(df_train_photo_ids, df_CNNcodes_train, on='photo_id')
del df_CNNcodes_train

In [ ]:
grouped = df_train_photo_biz_codes.groupby('business_id')
df = grouped.aggregate(lambda x: list(x))
df.drop('photo_id', axis=1, inplace=True)

In [ ]:
%%time
df["meanCNN"] = ""
    
for i,r in df.head(100).iterrows():
    CNNcode = np.array(r['CNNcode'])
    #CNNcode = duplicate_images(CNNcodes)
    meanCNN = CNNcode.mean(axis=0)
    df.loc[i, 'meanCNN'] = meanCNN

In [ ]:
%%time
df["meanCNN"] = ""
    
for i,r in df.iterrows():
    CNNcode = np.array(r['CNNcode'])
    #CNNcode = duplicate_images(CNNcodes)
    meanCNN = CNNcode.mean(axis=0)
    df.loc[i, 'meanCNN'] = meanCNN
        
#df.drop('CNNcode', axis=1, inplace = True)

In [ ]:
df.drop('CNNcode', axis=1, inplace = True)

In [ ]:
name = 'df_train_'+layer_name
df.to_pickle(name)

<h3> Test data

In [ ]:
df_CNNcodes_test =pd.DataFrame({'photo_id': [],'CNNcode': []})

# Extracting CNNcodes and photoids for test images
for f in os.listdir(pathcnn+"\\test"):
    photoids, CNNcodes = getCNNcodes(f, pathcnn+"\\test")
    df = pd.DataFrame({'photo_id': photoids,'CNNcode': CNNcodes})
    df_CNNcodes_test =pd.concat([df_CNNcodes_test, df])
    
df_CNNcodes_test = df_CNNcodes_test.convert_objects(convert_numeric = True)

In [ ]:
df_CNNcodes_test.head(5)

In [ ]:
df_test_photo_biz_codes = pd.merge(df_test_photo_ids, df_CNNcodes_test, on='photo_id')
del df_CNNcodes_test

In [ ]:
grouped = df_test_photo_biz_codes.groupby('business_id')
df = grouped.aggregate(lambda x: list(x))
df.drop('photo_id', axis=1, inplace=True)

In [ ]:
df.head(5)

In [ ]:
%%time
df["meanCNN"] = ""
    
for i,r in df.head(100).iterrows():
    CNNcode = np.array(r['CNNcode'])
    #CNNcode = duplicate_images(CNNcode)
    meanCNN = CNNcode.mean(axis=0)
    df.loc[i, 'meanCNN'] = meanCNN

In [ ]:
%%time
df["meanCNN"] = ""
    
for i,r in df.iterrows():
    CNNcode = np.array(r['CNNcode'])
    #CNNcode = duplicate_images(CNNcodes)
    meanCNN = CNNcode.mean(axis=0)
    df.loc[i, 'meanCNN'] = meanCNN
        
df.drop('CNNcode', axis=1, inplace = True)

In [ ]:
df.head(5)

In [ ]:
name = 'df_test_'+layer_name
df.to_pickle(name)